In [38]:
import pandas as pd
import os

In [39]:
path = "/content/drive/MyDrive/BIGDATA_STUDY/project_NLP/"
#파일 불러오기
df1 = pd.read_csv(path + "카페_validation.csv")
df2 = pd.read_csv(path + "카페_train.csv")
os.getcwd()

'/content'

In [40]:
#c-a , s-q 지우기
no_data1 = df1[ (df1['발화자'] == 's') & (df1['QA여부'] == 'q')].index
no_data2 = df1[ (df1['발화자'] == 'c') & (df1['QA여부'] == 'a')].index
df1_dr = df1.drop(no_data1)


no_data3 = df2[ (df2['발화자'] == 's') & (df2['QA여부'] == 'q')].index
no_data4 = df2[ (df2['발화자'] == 'c') & (df2['QA여부'] == 'a')].index
df2_dr = df2.drop(no_data3)

df1_dr.drop(no_data2, inplace = True)
df1_dr.reset_index(inplace = True, drop = True)

df2_dr.drop(no_data4, inplace = True)
df2_dr.reset_index(inplace = True, drop = True)


In [41]:
# qa 정렬대로 뽑아오기 
Q = []
A = []
Intent = []

for i in df1_dr.index:
  if df1_dr['QA여부'][i] == 'q' and df1_dr['QA여부'][i+1] == 'a':
    Q.append(df1_dr.loc[i]['발화문'])
    A.append(df1_dr.loc[i+1]['발화문'])
    Intent.append(df1_dr.loc[i]['인텐트'])

cafe_test_data= pd.DataFrame({'Q':Q,
                'A':A,
                'Label':Intent})

In [42]:
# qa 정렬대로 뽑아오기
Q = []
A = []
Intent = []

for i in df2_dr.index:
  if df2_dr['QA여부'][i] == 'q' and df2_dr['QA여부'][i+1] == 'a':
    Q.append(df2_dr.loc[i]['발화문'])
    A.append(df2_dr.loc[i+1]['발화문'])
    Intent.append(df2_dr.loc[i]['인텐트'])

data= pd.DataFrame({'Q':Q,
                'A':A,
                'Label':Intent})

In [43]:
# 중복 질문 제거
# 데이터: 72559 --> 중복 제거 후 데이터: 66041 
data.drop_duplicates(['Q'], inplace = True)
data.reset_index(inplace = True, drop = True)

In [44]:
# -------------------------------------------
# label 삭제
# 우리 카페에 필요한 데이터만 남기기
# -------------------------------------------

# 매장, 멤버십, 예약, 웹사이트, AS, 제품_추천, 제품_용도, 비교, 재고, 교환, 제품_가격 삭제
del_label = data[data['Label'].str.contains('매장|멤버십|예약|웹사이트|AS|제품_추천|제품_용도|비교|재고|교환|제품_가격')].index
data = data.drop(del_label)

data.reset_index(inplace = True, drop = True)

In [45]:
# 필요 없는 데이터 제거
extra_q = data[data['Q'].str.contains('설빙|베스킨라빈스|스타벅스|나무|단청|도마|그림|파인트|쿼터|패밀리|하프갤런|텀블러')].index
data = data.drop(extra_q)

extra_a = data[data['A'].str.contains('설빙|베스킨라빈스|스타벅스|나무|단청|도마|그림|파인트|쿼터|패밀리|하프갤런|텀블러')].index
data = data.drop(extra_a)

data.reset_index(inplace=True, drop=True)

In [46]:
# 날짜 묻는 질문/날짜로 대답하는 답변 제거
product_date_q = data[data['Q'].str.contains('1월|2월|3월|4월|5월|6월|7월|8월|9월|10월')].index
data = data.drop(product_date_q)

product_date_a = data[data['A'].str.contains('1월|2월|3월|4월|5월|6월|7월|8월|9월|10월')].index
data = data.drop(product_date_a)

data.reset_index(inplace=True, drop=True)


# 배달/배송 관련 정보 제거
delivery_q = data[data['Q'].str.contains('배달|택배')].index
data = data.drop(delivery_q)

delivery_a = data[data['A'].str.contains('배달|택배')].index
data = data.drop(delivery_a)

delivery = data[data['Label'].str.contains('배송')].index
data = data.drop(delivery)

data.reset_index(inplace=True, drop=True)

# 수거 제거
waste_q = data[data['Q'].str.contains('수거')].index
data = data.drop(waste_q)

waste_a = data[data['A'].str.contains('수거')].index
data = data.drop(waste_a)

data.reset_index(inplace=True, drop=True)

# 총 얼마예요? 제거
total_q = data[data['Q'].str.contains('총 얼마예요?')].index
data = data.drop(total_q)

data.reset_index(inplace=True, drop=True)

# 품절 제거
absence_q = data[data['Q'].str.contains('품절')].index
data = data.drop(absence_q)

absence_a = data[data['A'].str.contains('품절')].index
data = data.drop(absence_a)

data.reset_index(inplace=True, drop=True)

# 원두 삭제
coffee_q = data[data['Q'].str.contains('원두')].index
data = data.drop(coffee_q)

coffee_a = data[data['A'].str.contains('원두')].index
data = data.drop(coffee_a)

data.reset_index(inplace=True, drop=True)

In [47]:
# 재료 변경 불가 안내 문구 학습
data.loc[data['Q'].str.contains('빼|교환|바꾸|추가|바꿔|뺄'), 'A'] = '현재 재료 변경은 불가능합니다.'

# 디카페인 불가 안내 문구 학습
data.loc[data['Q'].str.contains('디카페인|디 카페인'), 'A'] = '현재 디카페인 음료는 제공되지 않습니다.'

# 제품 불량 문의 안내 문구 학습
data.loc[data['Label'].str.contains('제품_불량'), 'A'] = '죄송합니다. 카운터에서 확인 도와드리겠습니다.'

# 분할 결제 불가 안내 문구 학습
data.loc[data['Q'].str.contains('분할 결제|분할결제|나눠서 결제'), 'A'] = '현재 분할 결제가 불가능합니다. 다른 결제 수단을 선택해주세요.'

# 할인 불가 안내 문구 학습
data.loc[data['Q'].str.contains('할인|깎|깍'), 'A'] = '현재 할인은 불가합니다.'

# 칼로리 정보 불가능 안내 문구 학습
data.loc[data['Q'].str.contains('열량|칼로리'), 'A'] = '현재 칼로리 정보는 제공되지 않습니다. 카운터에 문의 바랍니다.'

# 행사 안내 문구 학습
data.loc[data['Q'].str.contains('행사|이벤트'), 'A'] = '현재 진행중인 행사가 없습니다. 다음 행사를 기다려주세요~^^'

In [48]:
# lst = list(cafe_train_data['Intent'])
# lst2 = [i.split('_')[0] if i.split('_')[0] != '제품' else i.split('_')[0] +'_'+ i.split('_')[2] for i in lst]Q

In [49]:
data

,Q,A,Label
0,빙수 위에 뿌려진 과일에서 쉰 냄새가 나는데 어떻게 하면 되죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
1,마카롱 주문했는데 마카롱이 엄청 딱딱해요.,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
2,와플 지금 받았는데 너무 눅눅해서 못먹겠어요 이거 어떻게 하면 되나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
3,케이크에 생크림 모양이 너무 변형된 것 같은데 어떻게 하나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
4,시루떡에서 왜 시큼한 냄새가 나는 거죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
...,...,...,...
37005,기프티콘으로 결제 가능한 거 맞죠?,"네, 기프티콘 결제 가능합니다.",결제_수단_확인
37006,카카오페이로 결제 된거 맞죠?,네 카카오페이로 결제되셨습니다.,결제_수단_확인
37007,모바일상품권도 결제할 수 있어요?,"죄송합니다, 모바일상품권은 사용이 어려우십니다.",결제_수단_확인
37008,기프티콘으로 음료 계산할 수 있는 거 맞죠?,"네, 기프티콘으로 음료 계산 가능합니다.",결제_수단_확인


#추가 데이터 병합

In [50]:
path = "/content/drive/MyDrive/BIGDATA_STUDY/project_NLP/"
#QA추가 csv 파일 불러오기
df_add = pd.read_csv(path + "cafe_add.csv")
os.getcwd()

'/content'

In [51]:
df_add

,Q,A,Label
0,아메리카노 얼마에요?,아메리카노 가격은 3500원 입니다.,제품_가격_질문
1,따뜻한 아메리카노 얼마에요?,따뜻한 아메리카노 가격은 3500원 입니다.,제품_가격_질문
2,아이스 아메리카노 얼마에요?,아이스 아메리카노 가격은 3500원 입니다.,제품_가격_질문
3,아메리카노 주세요,"차가운 것으로 드릴까요?, 따뜻한 것으로 드릴까요?",주문_제품_요청
4,아메리카노 한 잔 주세요,"차가운 것으로 드릴까요?, 따뜻한 것으로 드릴까요?",주문_제품_요청
...,...,...,...
2164,마실 것 뭐가 있나요?,"음료 메뉴는 아메리카노, 카페라떼, 카페모카, 바닐라라떼, 콜드브루, 카라멜 마끼아...",제품_정보_질문
2165,안녕하세요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",주문_제품_확인
2166,저기요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",주문_제품_확인
2167,벌레가 나왔어요,죄송합니다. 카운터에서 확인 도와드리겠습니다.,주문_제품_확인


In [52]:
df_comb = pd.concat([data, df_add])

In [53]:
df_comb

,Q,A,Label
0,빙수 위에 뿌려진 과일에서 쉰 냄새가 나는데 어떻게 하면 되죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
1,마카롱 주문했는데 마카롱이 엄청 딱딱해요.,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
2,와플 지금 받았는데 너무 눅눅해서 못먹겠어요 이거 어떻게 하면 되나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
3,케이크에 생크림 모양이 너무 변형된 것 같은데 어떻게 하나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
4,시루떡에서 왜 시큼한 냄새가 나는 거죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
...,...,...,...
2164,마실 것 뭐가 있나요?,"음료 메뉴는 아메리카노, 카페라떼, 카페모카, 바닐라라떼, 콜드브루, 카라멜 마끼아...",제품_정보_질문
2165,안녕하세요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",주문_제품_확인
2166,저기요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",주문_제품_확인
2167,벌레가 나왔어요,죄송합니다. 카운터에서 확인 도와드리겠습니다.,주문_제품_확인


In [54]:
#################################
### label 항목 레이블, 원핫인코딩(의도분석에 이용)
#################################

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#레이블 인코딩
lab_encoder = LabelEncoder()
lab_encoder.fit(df_comb['Label'].unique())
lst = lab_encoder.classes_
label_list = dict(zip(lst, range(len(lst))))

print(label_list)    #원래값-숫자 목록

{'결제_방식_요청': 0, '결제_방식_질문': 1, '결제_방식_확인': 2, '결제_수단_요청': 3, '결제_수단_질문': 4, '결제_수단_확인': 5, '결제_시기_요청': 6, '결제_시기_질문': 7, '결제_시기_확인': 8, '결제_영수증_요청': 9, '결제_영수증_질문': 10, '결제_영수증_확인': 11, '결제_오류_요청': 12, '결제_오류_질문': 13, '결제_오류_확인': 14, '결제_일반_요청': 15, '결제_일반_질문': 16, '결제_일반_확인': 17, '결제_재결제_요청': 18, '결제_재결제_질문': 19, '결제_재결제_확인': 20, '결제_추가_요청': 21, '결제_추가_질문': 22, '결제_추가_확인': 23, '결제_취소_요청': 24, '결제_취소_질문': 25, '결제_취소_확인': 26, '결제_할인_요청': 27, '결제_할인_질문': 28, '결제_할인_확인': 29, '제품_가격_질문': 30, '제품_구성_요청': 31, '제품_구성_질문': 32, '제품_구성_확인': 33, '제품_날짜_요청': 34, '제품_날짜_질문': 35, '제품_날짜_확인': 36, '제품_방법_요청': 37, '제품_방법_질문': 38, '제품_방법_확인': 39, '제품_불량_요청': 40, '제품_불량_질문': 41, '제품_불량_확인': 42, '제품_소재_요청': 43, '제품_소재_질문': 44, '제품_소재_확인': 45, '제품_원산지_요청': 46, '제품_원산지_질문': 47, '제품_원산지_확인': 48, '제품_일반_요청': 49, '제품_일반_질문': 50, '제품_일반_확인': 51, '제품_입고_요청': 52, '제품_입고_질문': 53, '제품_입고_확인': 54, '제품_정보_요청': 55, '제품_정보_질문': 56, '제품_정보_확인': 57, '제품_품질_요청': 58, '제품_품질_질문': 59, '제품_품질_확인': 60, '주문_변경_요청': 61, '주문_변경_질

In [55]:
#레이블 데이터 적용
lab_in=lab_encoder.transform(df_comb['Label'])   # label 값
ori_in=lab_encoder.inverse_transform(lab_in)         # 원래 값
df_comb['Label'] = lab_in

In [56]:
df_comb

,Q,A,Label
0,빙수 위에 뿌려진 과일에서 쉰 냄새가 나는데 어떻게 하면 되죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
1,마카롱 주문했는데 마카롱이 엄청 딱딱해요.,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
2,와플 지금 받았는데 너무 눅눅해서 못먹겠어요 이거 어떻게 하면 되나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
3,케이크에 생크림 모양이 너무 변형된 것 같은데 어떻게 하나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
4,시루떡에서 왜 시큼한 냄새가 나는 거죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
...,...,...,...
2164,마실 것 뭐가 있나요?,"음료 메뉴는 아메리카노, 카페라떼, 카페모카, 바닐라라떼, 콜드브루, 카라멜 마끼아...",56
2165,안녕하세요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",72
2166,저기요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",72
2167,벌레가 나왔어요,죄송합니다. 카운터에서 확인 도와드리겠습니다.,72


#데이터 저장

In [57]:
df_comb.to_csv(path + 'ChatbotData.csv')

In [58]:
pd.read_csv(path +'ChatbotData.csv')

,Unnamed: 0,Q,A,Label
0,0,빙수 위에 뿌려진 과일에서 쉰 냄새가 나는데 어떻게 하면 되죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
1,1,마카롱 주문했는데 마카롱이 엄청 딱딱해요.,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
2,2,와플 지금 받았는데 너무 눅눅해서 못먹겠어요 이거 어떻게 하면 되나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
3,3,케이크에 생크림 모양이 너무 변형된 것 같은데 어떻게 하나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
4,4,시루떡에서 왜 시큼한 냄새가 나는 거죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
...,...,...,...,...
39174,2164,마실 것 뭐가 있나요?,"음료 메뉴는 아메리카노, 카페라떼, 카페모카, 바닐라라떼, 콜드브루, 카라멜 마끼아...",56
39175,2165,안녕하세요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",72
39176,2166,저기요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",72
39177,2167,벌레가 나왔어요,죄송합니다. 카운터에서 확인 도와드리겠습니다.,72


In [59]:
cafe_test_data.to_csv(path + 'cafe_qa_validation.csv')

In [60]:
pd.read_csv(path +'cafe_qa_validation.csv')

,Unnamed: 0,Q,A,Label
0,0,케이크에서 약간 시큼한 맛이 나는데 상한건가요?,"죄송합니다, 새걸로 바꿔드릴게요.",제품_불량_질문
1,1,주스 안에서 돌 같은게 나왔는데 확인 좀 해주시겠어요?,제조과정에서 문제가 있었던 걸로 추측되네요.,제품_불량_질문
2,2,이 에그타르트 가운데가 너무 날계란물인 것 같은데 원래 이런 거예요?,안쪽이 약간 덜 익은 제품이 나갔나봐요. 다른 걸로 드릴게요.,제품_불량_질문
3,3,이 딸기 케이크 사이즈가 잘못 표기된 것 같은데요.,바로 확인해서 알려드리겠습니다.,제품_불량_질문
4,4,케이크 주문했는데 생크림 상한 것 같습니다.,죄송합니다. 환불을 원하시면 바로 환불해 드릴게요.,제품_불량_질문
...,...,...,...,...
9517,9517,포인트 차감으로 프레첼 머랭 쿠키 구매할 수 있는 거죠?,"네, 포인트 차감으로 프레첼 머랭 쿠키를 구매하실 수 있습니다.",결제_수단_확인
9518,9518,지난주에 매장에서 보내준 생일 쿠폰이 있어요.,"네, 1년 안에 쿠폰 쓰시면 됩니다.",결제_수단_확인
9519,9519,카카오톡 기프티콘이랑 롯데 상품권 동시에 사용해도 되는 거죠?,"네, 기프티콘과 상품권 동시 사용 가능합니다.",결제_수단_확인
9520,9520,기프티콘 적용된 것 맞나요?,네. 잘 적용되었습니다.,결제_수단_확인
